In [1]:
!pip install torch==1.9 torchtext==0.10

In [2]:
!pip install datasets

In [3]:
import math
import time

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

import tqdm

import datasets

In [4]:
print(torch.__version__)
print(torchtext.__version__)

1.9.0+cu102
0.10.0


In [5]:
dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')

Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)


In [6]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [7]:
dataset['train'][0]

{'text': ''}

In [8]:
dataset['train'][1]

{'text': ' = Valkyria Chronicles III = \n'}

In [9]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

In [10]:
tokenizer('hello world how are you?')

['hello', 'world', 'how', 'are', 'you', '?']

In [11]:
tokenizer(dataset['train'][1]['text'])

['=', 'valkyria', 'chronicles', 'iii', '=']

In [12]:
def tokenize_data(example, tokenizer):
    tokens = {'tokens': tokenizer(example['text'])}
    return tokens

In [13]:
tokenized_dataset = dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-ad820ce433d49621.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-677f66b38513724d.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-3eb0562c08f69ed1.arrow


In [14]:
tokenized_dataset['train'][1]

{'tokens': ['=', 'valkyria', 'chronicles', 'iii', '=']}

In [15]:
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['train']['tokens'],
                                                  min_freq=3)

In [16]:
vocab.get_itos()[:10]

['the', ',', '.', 'of', 'and', 'in', 'to', 'a', '=', 'was']

In [17]:
len(vocab)

29471

In [18]:
'hello' in vocab

False

In [19]:
vocab.insert_token('<unk>', 0)

In [20]:
vocab.get_itos()[:10]

['<unk>', 'the', ',', '.', 'of', 'and', 'in', 'to', 'a', '=']

In [21]:
vocab.set_default_index(0)

In [22]:
vocab['hello']

0

In [23]:
vocab.insert_token('<eos>', 1)

In [24]:
vocab.get_itos()[:10]

['<unk>', '<eos>', 'the', ',', '.', 'of', 'and', 'in', 'to', 'a']

In [25]:
def get_data(dataset, vocab, batch_size):
    data = []
    for example in dataset:
        if example['tokens']:
            tokens = example['tokens'].append('<eos>')
            tokens = [vocab[token] for token in example['tokens']]
            data.extend(tokens)
    data = torch.LongTensor(data)
    n_batches = data.shape[0] // batch_size
    data = data.narrow(0, 0, n_batches * batch_size)
    data = data.view(batch_size, -1)
    return data

In [26]:
batch_size = 128

train_data = get_data(tokenized_dataset['train'], vocab, batch_size)

In [27]:
train_data.shape

torch.Size([128, 16214])

In [28]:
valid_data = get_data(tokenized_dataset['validation'], vocab, batch_size)
test_data = get_data(tokenized_dataset['test'], vocab, batch_size)

In [29]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, dropout_rate, tie_weights):
        super().__init__()
        self.vocab_size = vocab_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout_rate, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

        init_range = 0.1
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

        if tie_weights:
            assert embedding_dim == hidden_dim, 'If tying weights then embedding_dim must equal hidden_dim'
            self.embedding.weight = self.fc.weight
        self.dropout = nn.Dropout(dropout_rate)
    
    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell

    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

    def forward(self, input, hidden):
        # input = [batch size, seq len]
        # hidden = [n layers, batch size, hidden dim]
        embedding = self.dropout(self.embedding(input))
        # embedding = [batch size, seq len, embedding dim]
        output, hidden = self.lstm(embedding, hidden)
        # output = [batch size, seq len, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        output = self.dropout(output)
        output = self.fc(output)
        # output = [batch size, seq len, vocab size]
        return output, hidden

In [30]:
vocab_size = len(vocab)
embedding_dim = 1024
hidden_dim = 1024
n_layers = 2
dropout_rate = 0.5
tie_weights = True

model = LSTM(vocab_size, embedding_dim, hidden_dim, n_layers, dropout_rate, tie_weights)

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 47,003,425 trainable parameters


In [32]:
optimizer = optim.Adam(model.parameters())

In [33]:
criterion = nn.CrossEntropyLoss()

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [35]:
model = model.to(device)
criterion = criterion.to(device)

In [36]:
def train(model, data, optimizer, criterion, batch_size, max_seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    n_tokens = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    
    for offset in tqdm.tqdm(range(0, n_tokens - 1, max_seq_len)):
        optimizer.zero_grad()
        input, target, seq_len = get_batch(data, max_seq_len, n_tokens, offset)
        input = input.to(device)
        target = target.to(device)
        # input = [batch size, seq len]
        # target = [batch size, seq len]
        hidden = model.detach_hidden(hidden)
        # hidden = [n layers, batch size, hidden dim]
        output, hidden = model(input, hidden)
        # output = [batch size, seq len, vocab size]
        # hidden = [n layers, batch size, hidden dim]
        output = output.reshape(-1, model.vocab_size)
        target = target.reshape(-1)
        # output = [batch size * seq len, vocab size]
        # target = [batch size * seq len]
        loss = criterion(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / n_tokens

In [37]:
def get_batch(data, max_seq_len, n_tokens, offset):
    seq_len = min(max_seq_len, n_tokens - offset - 1)
    input = data[:, offset:offset+seq_len]
    target = data[:, offset+1:offset+seq_len+1]
    return input, target, seq_len

In [38]:
def evaluate(model, data, criterion, batch_size, max_seq_len, device):

    epoch_loss = 0
    model.eval()
    n_tokens = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for offset in tqdm.tqdm(range(0, n_tokens - 1, max_seq_len)):
            input, target, seq_len = get_batch(data, max_seq_len, n_tokens, offset)
            input = input.to(device)
            target = target.to(device)
            # input = [batch size, seq len]
            # target = [batch size, seq len]
            hidden = model.detach_hidden(hidden)
            # hidden = [n layers, batch size, hidden dim]
            output, hidden = model(input, hidden)
            # output = [batch size, seq len, vocab size]
            # hidden = [n layers, batch size, hidden dim]
            output = output.reshape(-1, model.vocab_size)
            target = target.reshape(-1)
            # output = [batch size * seq len, vocab size]
            # target = [batch size * seq len]
            loss = criterion(output, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / n_tokens

In [39]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

In [41]:
n_epochs = 50
max_seq_len = 50
clip = 0.25

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()

    train_loss = train(model, train_data, optimizer, criterion, batch_size, max_seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, max_seq_len, device)
    
    lr_scheduler.step(valid_loss)

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm_lm.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(train_loss, valid_loss)
    try:
        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')
    except:
        pass

  0%|          | 0/325 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 1m 29s
6.319018106300587 5.519182275769846
	Train Perplexity: 555.028
	Valid Perplexity: 249.431


  0%|          | 0/325 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 1m 29s
5.504364474569368 5.166163277232422
	Train Perplexity: 245.762
	Valid Perplexity: 175.241


  0%|          | 0/325 [00:00<?, ?it/s]

...

Epoch: 28 | Epoch Time: 1m 29s
3.6615614594021575 4.64130736887455
	Train Perplexity: 38.922
	Valid Perplexity: 103.680


  0%|          | 0/325 [00:00<?, ?it/s]

Epoch: 29 | Epoch Time: 1m 29s
3.6669943484275396 4.640843982122979
	Train Perplexity: 39.134
	Valid Perplexity: 103.632


 25%|██▌       | 82/325 [00:21<01:05,  3.73it/s]

KeyboardInterrupt: ignored

In [42]:
model.load_state_dict(torch.load('lstm_lm.pt'))

test_loss = evaluate(model, test_data, criterion, batch_size, max_seq_len, device)

print(f'Test Perplexity: {math.exp(test_loss):.3f}')


100%|██████████| 39/39 [00:03<00:00, 11.74it/s]

Test Perplexity: 99.358


In [43]:
def generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device):

    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(n_gen_tokens):
            input = torch.LongTensor([indices]).to(device)
            output, hidden = model(input, hidden)
            probs = torch.softmax(output[:, -1] / temperature, dim=-1) 
            prediction = torch.multinomial(probs, num_samples=1).item()
            indices.append(prediction)

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [50]:
prompt = 'my favourite color'
n_gen_tokens = 25
temperature = 1.0

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device)

In [51]:
generation

['my',
 'favourite',
 'color',
 ',',
 'genius',
 'or',
 'seductive',
 ',',
 'has',
 'to',
 'be',
 'good',
 'for',
 'the',
 'first',
 'time',
 'or',
 'a',
 'classic',
 'christmas',
 'series',
 ',',
 'as',
 'well',
 'as',
 'the',
 'observer',
 'below']